In [2]:
import pandas as pd

In [5]:
# Load just a few columns for efficiency
noteevents = pd.read_csv("NOTEEVENTS.csv", usecols=["SUBJECT_ID", "HADM_ID", "CHARTDATE", "CATEGORY", "TEXT"])

# Filter for discharge summaries
discharge_notes = noteevents[noteevents["CATEGORY"] == "Discharge summary"].copy()

# Preview
discharge_notes.head()

,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,TEXT
0,22532,167853.0,2151-08-04,Discharge summary,Admission Date: [**2151-7-16**] Dischar...
1,13702,107527.0,2118-06-14,Discharge summary,Admission Date: [**2118-6-2**] Discharg...
2,13702,167118.0,2119-05-25,Discharge summary,Admission Date: [**2119-5-4**] D...
3,13702,196489.0,2124-08-18,Discharge summary,Admission Date: [**2124-7-21**] ...
4,26880,135453.0,2162-03-25,Discharge summary,Admission Date: [**2162-3-3**] D...


In [11]:
import re
from collections import Counter

def get_all_headers(text):
    # Find all lines that look like "SECTION NAME:"
    return re.findall(r'\n([A-Z \-]+):', text)

all_headers = discharge_notes['TEXT'].apply(get_all_headers)
header_counts = Counter([header for sublist in all_headers for header in sublist])

# Print top 30 most common headers
for header, count in header_counts.most_common(30):
    print(f"{header}: {count}")


IMPRESSION: 34143
HEENT: 30163
T: 19602
D: 18621
CV: 17766
HISTORY OF PRESENT ILLNESS: 12109
HOSPITAL COURSE: 10640
PAST MEDICAL HISTORY: 10608
FINDINGS: 9270
ALLERGIES: 8914
VS: 8855
PHYSICAL EXAMINATION: 8608
DISCHARGE DIAGNOSES: 8038
DISCHARGE MEDICATIONS: 7036
SOCIAL HISTORY: 6982
NEURO: 5685
ABD: 5644
CONDITION ON DISCHARGE: 5406
NECK: 5392
GEN: 5182
SKIN: 5065
EXT: 5048
LUNGS: 4977
ABDOMEN: 4730
GU: 4681
GENERAL: 4443
MEDICATIONS ON ADMISSION: 4105
DISCHARGE STATUS: 3947
EXTREMITIES: 3915
MEDICATIONS: 3877


In [16]:
RELEVANT_HEADERS = {
    'RADIOLOGIC STUDIES',
    'HEAD CT',
    'ABDOMINAL CT',
    'IMPRESSION',
    'FINDINGS',
    'HISTORY OF PRESENT ILLNESS',
    'HOSPITAL COURSE',
    'PHYSICAL EXAMINATION'
}


def extract_sections_flexible(text):
    sections = {}
    current_section = None
    lines = text.split('\n')

    for line in lines:
        line_stripped = line.strip()
        if re.match(r'^[A-Z][A-Z \-]*:$', line_stripped):  # Match lines like "HEAD CT:"
            current_section = line_stripped[:-1].strip()
            sections[current_section] = []
        elif current_section:
            sections[current_section].append(line_stripped)

    # Keep only sections we care about
    kept_content = []
    for key in sections:
        if any(rel in key for rel in RELEVANT_HEADERS):
            kept_content.extend(sections[key])

    return ' '.join(kept_content).strip()


In [17]:
discharge_notes['TEXT_CLEAN'] = discharge_notes['TEXT'].apply(extract_relevant_sections)

In [18]:
print("🔹 ORIGINAL NOTE:\n")
print(discharge_notes['TEXT'].iloc[0][:1500])

print("\n🔹 CLEANED NOTE (Filtered Sections Only):\n")
print(discharge_notes['TEXT_CLEAN'].iloc[0][:1500])

🔹 ORIGINAL NOTE:

Admission Date:  [**2151-7-16**]       Discharge Date:  [**2151-8-4**]


Service:
ADDENDUM:

RADIOLOGIC STUDIES:  Radiologic studies also included a chest
CT, which confirmed cavitary lesions in the left lung apex
consistent with infectious process/tuberculosis.  This also
moderate-sized left pleural effusion.

HEAD CT:  Head CT showed no intracranial hemorrhage or mass
effect, but old infarction consistent with past medical
history.

ABDOMINAL CT:  Abdominal CT showed lesions of
T10 and sacrum most likely secondary to osteoporosis. These can
be followed by repeat imaging as an outpatient.



                            [**First Name8 (NamePattern2) **] [**First Name4 (NamePattern1) 1775**] [**Last Name (NamePattern1) **], M.D.  [**MD Number(1) 1776**]

Dictated By:[**Hospital 1807**]
MEDQUIST36

D:  [**2151-8-5**]  12:11
T:  [**2151-8-5**]  12:21
JOB#:  [**Job Number 1808**]


🔹 CLEANED NOTE (Filtered Sections Only):

Radiologic studies also included a chest
CT, whic